<a href="https://colab.research.google.com/github/skhetrapal/kaggle_projects/blob/main/Kaggle_loan_approval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

train_path = '/content/drive/My Drive/Colab data/Kaggle/loan_approval_pred/train.csv'
test_path = '/content/drive/My Drive/Colab data/Kaggle/loan_approval_pred/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [ ]:
train_df.isna().sum()

,0
id,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [ ]:
test_df.isna().sum()

,0
id,0
person_age,0
person_income,0
person_home_ownership,0
person_emp_length,0
loan_intent,0
loan_grade,0
loan_amnt,0
loan_int_rate,0
loan_percent_income,0


In [ ]:
train_df['loan_status']

,loan_status
0,0
1,0
2,0
3,0
4,0
...,...
58640,0
58641,1
58642,1
58643,0


In [ ]:
train_df['cb_person_default_on_file'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
train_df['loan_intent'].unique()

array(['EDUCATION', 'MEDICAL', 'PERSONAL', 'VENTURE', 'DEBTCONSOLIDATION',
       'HOMEIMPROVEMENT'], dtype=object)

In [ ]:
train_df['person_home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

In [ ]:
train_df['loan_grade'].unique()

array(['B', 'C', 'A', 'D', 'E', 'F', 'G'], dtype=object)

In [ ]:
test_df['loan_status'] = 0

In [ ]:
test_df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2,0
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4,0
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2,0
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7,0
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4,0


In [ ]:
df = pd.concat([train_df, test_df],axis = 0)

In [ ]:
df.shape

(97743, 13)

In [ ]:
train_df.shape[0], test_df.shape[0]

(58645, 39098)

In [ ]:
df.shape[0]==train_df.shape[0] + test_df.shape[0]

True

In [ ]:
def preprocess(df):
  # Create a mapping dictionary
  mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F':5, 'G':6}
  # Map the values in column 'A' using the mapping dictionary
  df['loan_grade'] = df['loan_grade'].map(mapping)

  mapping = {'N': 0, 'Y': 1}
  df['cb_person_default_on_file'] = df['cb_person_default_on_file'].map(mapping)

  df_encoded = pd.get_dummies(df, columns=['loan_intent', 'person_home_ownership'])

  df_train = df_encoded[:train_df.shape[0]]
  df_test = df_encoded[train_df.shape[0]:]
  df_test.drop('loan_status', axis=1, inplace=True)

  return df_train, df_test

In [ ]:
train_df, test_df = preprocess(df)

<ipython-input-15-fd29ef6f4f42>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop('loan_status', axis=1, inplace=True)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Y = train_df['loan_status']
X = train_df.drop('loan_status', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Initialize and train different classifiers

model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Logistic Regression AUC: {roc_auc}")

model_2 = RandomForestClassifier(random_state=42)
model_2.fit(X_train, y_train)
y_pred_rf = model_2.predict(X_test)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"Random Forest AUC: {roc_auc_rf}")

model_3 = GradientBoostingClassifier(random_state=42)
model_3.fit(X_train, y_train)
y_pred_gb = model_3.predict(X_test)
roc_auc_gb = roc_auc_score(y_test, y_pred_gb)
print(f"Gradient Boosting AUC: {roc_auc_gb}")

# model_4 = SVC(probability=True, random_state=42) #probability=True for roc_auc_score
# model_4.fit(X_train, y_train)
# y_pred_svc = model_4.predict(X_test)
# roc_auc_svc = roc_auc_score(y_test, y_pred_svc)
# print(f"SVC AUC: {roc_auc_svc}")

model_5 = KNeighborsClassifier()
model_5.fit(X_train, y_train)
y_pred_knn = model_5.predict(X_test)
roc_auc_knn = roc_auc_score(y_test, y_pred_knn)
print(f"KNN AUC: {roc_auc_knn}")

model_6 = DecisionTreeClassifier(random_state=42)
model_6.fit(X_train, y_train)
y_pred_dt = model_6.predict(X_test)
roc_auc_dt = roc_auc_score(y_test, y_pred_dt)
print(f"Decision Tree AUC: {roc_auc_dt}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression AUC: 0.6927744759447858
Random Forest AUC: 0.8551187494618984
Gradient Boosting AUC: 0.8542265119284395
KNN AUC: 0.6555541695893715
Decision Tree AUC: 0.8314957373892204


In [ ]:
test_pred = model_2.predict(test_df)

In [ ]:
test_pred

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
test_pred.shape

(39098,)

In [ ]:
# convert array to df

import numpy as np
# Assuming 'test_pred' is your NumPy array
test_pred_df = pd.DataFrame(test_pred, columns=['loan_status'])
print(test_pred_df.head())

   loan_status
0            1
1            0
2            1
3            0
4            0


In [ ]:
test_df['id'].head()

,id
0,58645
1,58646
2,58647
3,58648
4,58649


In [ ]:
result_df = pd.concat([test_df['id'], test_pred_df],axis = 1)

In [ ]:
result_df.head()

,id,loan_status
0,58645,1
1,58646,0
2,58647,1
3,58648,0
4,58649,0


In [ ]:
result_df.to_csv('output.csv', index=False)